<a href="https://colab.research.google.com/github/tc10767557/AI/blob/main/HW3/Problem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Teagan Clark

Dr. Forouraghi

Artificial Intelligence

21 October 2025

#Assignment 3
##Problem 3

Coded in PyCharm, then copied to Colab to submit to GitHub. The code does not work in Colab! Please use PyCharm (or another IDE) to run this code.

#####Part 1

In [16]:
# Part 1

print("α   β   Algorithm Type       Observed Behavior")
print("---------------------------------------------------------------------------------------")
print("1   1   Standard A*          Balanced exploration and optimal path")
print("0   1   Greedy Best-First    Moves directly toward goal, short runtime, but not optimal")
print("1   3   Weighted A*          Also goal-biased, similar longer path")

α   β   Algorithm Type       Observed Behavior
---------------------------------------------------------------------------------------
1   1   Standard A*          Balanced exploration and optimal path
0   1   Greedy Best-First    Moves directly toward goal, short runtime, but not optimal
1   3   Weighted A*          Also goal-biased, similar longer path


#####Part 2

In [17]:
# imports
import tkinter as tk
from queue import PriorityQueue
import math, random

In [18]:
# cell to store f(), g() and h() values
class Cell:
    def __init__(self, x, y, is_wall=False):
        self.x = x
        self.y = y
        self.is_wall = is_wall
        self.g = float("inf")
        self.h = 0
        self.f = float("inf")
        self.parent = None

    def __lt__(self, other):
        return self.f < other.f

In [19]:
# MazeGame to visualize the pathfinding algorithms
class MazeGame:
    def __init__(self, root, maze, algorithm="A*", alpha=1, beta=1):
        self.root = root
        self.maze = maze
        self.algorithm = algorithm   # "A*" or "Greedy"
        self.alpha = alpha
        self.beta = beta

        self.rows = len(maze)
        self.cols = len(maze[0])

        # start and goal positions
        self.agent_pos = (13, 4)
        self.goal_pos = (21, 20)

        self.cells = [[Cell(x, y, maze[x][y] == 1) for y in range(self.cols)] for x in range(self.rows)]

        # initialize start cell
        start = self.cells[self.agent_pos[0]][self.agent_pos[1]]
        start.g = 0
        start.h = self.heuristic(self.agent_pos)
        start.f = self.alpha * start.g + self.beta * start.h if algorithm == "A*" else self.beta * start.h

        # visualization settings
        self.cell_size = 20
        self.canvas = tk.Canvas(root, width=self.cols * self.cell_size,
                                height=self.rows * self.cell_size, bg='white')
        self.canvas.pack()

        self.draw_maze()
        self.find_path()


    # Euclidean distance heuristic
    def heuristic(self, pos):
        dx = pos[0] - self.goal_pos[0]
        dy = pos[1] - self.goal_pos[1]
        return math.sqrt(dx * dx + dy * dy)


    # pathfinding (Weighted A* or Greedy Best-First)
    def find_path(self):
        open_set = PriorityQueue()
        open_set.put((0, self.agent_pos))

        while not open_set.empty():
            _, current_pos = open_set.get()
            current_cell = self.cells[current_pos[0]][current_pos[1]]

            if current_pos == self.goal_pos:
                self.reconstruct_path()
                break

            # 8 possible movement directions
            directions = [
                (0, 1), (0, -1), (1, 0), (-1, 0),
                (1, 1), (1, -1), (-1, 1), (-1, -1)
            ]
            random.shuffle(directions)

            for dx, dy in directions:
                new_pos = (current_pos[0] + dx, current_pos[1] + dy)
                if (0 <= new_pos[0] < self.rows and 0 <= new_pos[1] < self.cols
                    and not self.cells[new_pos[0]][new_pos[1]].is_wall):

                    new_cell = self.cells[new_pos[0]][new_pos[1]]

                    # diagonal cost is sqrt(2), straight cost is 1
                    move_cost = math.sqrt(2) if dx != 0 and dy != 0 else 1
                    tentative_g = current_cell.g + move_cost
                    h = self.heuristic(new_pos)

                    if self.algorithm == "Greedy":
                        f = self.beta * h
                    else:
                        f = self.alpha * tentative_g + self.beta * h

                    if f < new_cell.f:
                        new_cell.g = tentative_g
                        new_cell.h = h
                        new_cell.f = f
                        new_cell.parent = current_cell
                        open_set.put((new_cell.f, new_pos))\
    # draw maze
    def draw_maze(self):
        for x in range(self.rows):
            for y in range(self.cols):
                color = 'maroon' if self.maze[x][y] == 1 else 'white'
                self.canvas.create_rectangle(
                    y * self.cell_size, x * self.cell_size,
                    (y + 1) * self.cell_size, (x + 1) * self.cell_size, fill=color)

        # draw start and goal
        sx, sy = self.agent_pos
        gx, gy = self.goal_pos
        self.canvas.create_rectangle(
            sy * self.cell_size, sx * self.cell_size,
            (sy + 1) * self.cell_size, (sx + 1) * self.cell_size, fill='blue')
        self.canvas.create_rectangle(
            gy * self.cell_size, gx * self.cell_size,
            (gy + 1) * self.cell_size, (gx + 1) * self.cell_size, fill='red')

        self.canvas.create_text(
            self.cols * self.cell_size / 2,
            self.rows * self.cell_size + 10,
            text=f"{self.algorithm} (α={self.alpha}, β={self.beta})",
            anchor="n", font=("Purisa", 14, "bold"), fill="black")


    # trace final path
    def reconstruct_path(self):
        current_cell = self.cells[self.goal_pos[0]][self.goal_pos[1]]
        while current_cell.parent:
            x, y = current_cell.x, current_cell.y
            self.canvas.create_rectangle(
                y * self.cell_size, x * self.cell_size,
                (y + 1) * self.cell_size, (x + 1) * self.cell_size,
                fill='green')
            current_cell = current_cell.parent

In [20]:
# maze setup (25x25)
maze = [
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
]

In [20]:
# GUI for multiple algorithm settings
root1 = tk.Tk()
root1.title("Weighted A* (β=1)")
game_astar = MazeGame(root1, maze, algorithm="A*", alpha=1, beta=1)

root2 = tk.Toplevel()
root2.title("Weighted A* (β=3)")
game_astar_weighted = MazeGame(root2, maze, algorithm="A*", alpha=1, beta=3)

root3 = tk.Toplevel()
root3.title("Greedy Best-First Search")
game_greedy = MazeGame(root3, maze, algorithm="Greedy")

tk.mainloop()